# Preprocessing Traffic Data

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium

In [5]:
# Read traffic data
tdata = pd.read_csv('../data/landing/external/NYC_traffic.csv')
tdata.head(10)

,id,segmentid,roadway_name,from,to,direction,date,_12_00_1_00_am,_1_00_2_00am,_2_00_3_00am,...,_2_00_3_00pm,_3_00_4_00pm,_4_00_5_00pm,_5_00_6_00pm,_6_00_7_00pm,_7_00_8_00pm,_8_00_9_00pm,_9_00_10_00pm,_10_00_11_00pm,_11_00_12_00am
0,1,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-09T00:00:00.000,20.0,10.0,11.0,...,104.0,105.0,147.0,120.0,91.0,83.0,74.0,49.0,42.0,42.0
1,2,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-10T00:00:00.000,21.0,16.0,8.0,...,102.0,98.0,133.0,131.0,95.0,73.0,70.0,63.0,42.0,35.0
2,3,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-11T00:00:00.000,27.0,14.0,6.0,...,115.0,115.0,130.0,143.0,106.0,89.0,68.0,64.0,56.0,43.0
3,4,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-12T00:00:00.000,22.0,7.0,7.0,...,71.0,127.0,122.0,144.0,122.0,76.0,64.0,58.0,64.0,43.0
4,5,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-13T00:00:00.000,31.0,17.0,7.0,...,113.0,126.0,133.0,135.0,102.0,106.0,58.0,58.0,55.0,54.0
5,6,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-14T00:00:00.000,42.0,27.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,SB,2012-01-09T00:00:00.000,27.0,12.0,12.0,...,165.0,197.0,152.0,174.0,128.0,95.0,87.0,73.0,57.0,42.0
7,8,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,SB,2012-01-10T00:00:00.000,26.0,16.0,11.0,...,192.0,180.0,161.0,171.0,120.0,96.0,90.0,70.0,63.0,49.0
8,9,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,SB,2012-01-11T00:00:00.000,32.0,16.0,8.0,...,178.0,194.0,168.0,160.0,143.0,114.0,78.0,64.0,49.0,45.0
9,10,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,SB,2012-01-12T00:00:00.000,24.0,12.0,7.0,...,171.0,184.0,157.0,167.0,148.0,112.0,86.0,77.0,63.0,51.0


In [6]:
tdata.tail(10)

,id,segmentid,roadway_name,from,to,direction,date,_12_00_1_00_am,_1_00_2_00am,_2_00_3_00am,...,_2_00_3_00pm,_3_00_4_00pm,_4_00_5_00pm,_5_00_6_00pm,_6_00_7_00pm,_7_00_8_00pm,_8_00_9_00pm,_9_00_10_00pm,_10_00_11_00pm,_11_00_12_00am
42746,372,69698,GERARD AVENUE,East 150th Street,East 151st Street,NB,2020-10-25T00:00:00.000,73.0,59.0,28.0,...,175.0,222.0,223.0,211.0,195.0,178.0,127.0,110.0,87.0,67.0
42747,373,35832,WEST 49th STREET,Rockefeller Plaza,6th Avenue,WB,2020-11-14T00:00:00.000,102.0,71.0,47.0,...,109.0,136.0,144.0,150.0,144.0,162.0,181.0,165.0,175.0,168.0
42748,373,35832,WEST 49th STREET,Rockefeller Plaza,6th Avenue,WB,2020-11-15T00:00:00.000,145.0,95.0,52.0,...,147.0,193.0,193.0,172.0,162.0,184.0,188.0,138.0,118.0,100.0
42749,373,35832,WEST 49th STREET,Rockefeller Plaza,6th Avenue,WB,2020-11-16T00:00:00.000,55.0,43.0,49.0,...,142.0,108.0,146.0,139.0,174.0,184.0,139.0,128.0,109.0,113.0
42750,373,35832,WEST 49th STREET,Rockefeller Plaza,6th Avenue,WB,2020-11-17T00:00:00.000,69.0,45.0,26.0,...,162.0,159.0,164.0,188.0,194.0,179.0,139.0,139.0,114.0,103.0
42751,373,35832,WEST 49th STREET,Rockefeller Plaza,6th Avenue,WB,2020-11-18T00:00:00.000,68.0,63.0,31.0,...,152.0,177.0,162.0,164.0,183.0,173.0,143.0,113.0,127.0,133.0
42752,373,35832,WEST 49th STREET,Rockefeller Plaza,6th Avenue,WB,2020-11-19T00:00:00.000,71.0,59.0,42.0,...,166.0,162.0,187.0,182.0,180.0,190.0,162.0,155.0,234.0,166.0
42753,373,35832,WEST 49th STREET,Rockefeller Plaza,6th Avenue,WB,2020-11-20T00:00:00.000,111.0,79.0,54.0,...,157.0,186.0,170.0,158.0,194.0,196.0,222.0,174.0,182.0,178.0
42754,373,35832,WEST 49th STREET,Rockefeller Plaza,6th Avenue,WB,2020-11-21T00:00:00.000,129.0,95.0,63.0,...,139.0,147.0,153.0,183.0,163.0,184.0,183.0,169.0,187.0,211.0
42755,373,35832,WEST 49th STREET,Rockefeller Plaza,6th Avenue,WB,2020-11-22T00:00:00.000,159.0,109.0,77.0,...,151.0,176.0,152.0,146.0,181.0,177.0,160.0,162.0,147.0,137.0
